In [1]:
import git, os
git_repo = git.Repo(os.getcwd(), search_parent_directories=True)
git_root = git_repo.git.rev_parse("--show-toplevel")
os.chdir(f'{git_root}/src')
from data_handler import DataHandler

C:\Users\Alex\OneDrive\Documents\GitHub\UFC_Prediction_2022\src\fight_stat_helpers.py:15: DtypeWarning: Columns (8,73) have mixed types. Specify dtype option on import or set low_memory=False.
  fight_hist = pd.read_csv('content/data/processed/ufc_fights.csv')
C:\Users\Alex\OneDrive\Documents\GitHub\UFC_Prediction_2022\src\fight_stat_helpers.py:17: DtypeWarning: Columns (15,37,38,102,103) have mixed types. Specify dtype option on import or set low_memory=False.
  ufc_fights_crap = pd.read_csv('content/data/processed/ufc_fights_crap.csv')


In [2]:
dh = DataHandler()

C:\Users\Alex\OneDrive\Documents\GitHub\UFC_Prediction_2022\src\data_handler\data_handler.py:84: DtypeWarning: Columns (15,34) have mixed types. Specify dtype option on import or set low_memory=False.
  self.csv_data = {key : pd.read_csv(self.csv_filepaths[key], sep=',') for key in self.csv_filepaths.keys()}
C:\Users\Alex\OneDrive\Documents\GitHub\UFC_Prediction_2022\src\data_handler\data_handler.py:84: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  self.csv_data = {key : pd.read_csv(self.csv_filepaths[key], sep=',') for key in self.csv_filepaths.keys()}
C:\Users\Alex\OneDrive\Documents\GitHub\UFC_Prediction_2022\src\data_handler\data_handler.py:84: DtypeWarning: Columns (15,37,38,102,103) have mixed types. Specify dtype option on import or set low_memory=False.
  self.csv_data = {key : pd.read_csv(self.csv_filepaths[key], sep=',') for key in self.csv_filepaths.keys()}
C:\Users\Alex\OneDrive\Documents\GitHub\UFC_Prediction_2022\sr

In [3]:
prediction_history = dh.get('prediction_history', filetype='json')

In [ ]:
prediction_history.head()

In [ ]:
prediction_history.columns

In [4]:
fighter_bookie_columns = [f'fighter {bookie}' for bookie in dh.bookies]
opponent_bookie_columns = [f'opponent {bookie}' for bookie in dh.bookies]

In [5]:
# make mask on prediction history so that fighter bookie columns are not empty strings 
has_fighter_bookie_odds_mask = prediction_history[fighter_bookie_columns].apply(lambda x: x != '').any(axis=1)

In [6]:
from fight_stat_helpers import get_kelly_bet_from_ev_and_dk_odds

In [7]:
# for each row in prediction history, find the bookie with the best odds for the fighter
for i, row in prediction_history[has_fighter_bookie_odds_mask].iterrows():
    predicted_fighter_odds = row['predicted fighter odds']
    predicted_opponent_odds = row['predicted opponent odds']
    
    bookies_with_fighter_odds = {}    
    bookies_with_opponent_odds = {}
    for bookie in dh.bookies:
        fighter_col = f'fighter {bookie}'
        # if the fighter bookie odds are empty, set them to the predicted fighter odds
        if row[fighter_col] == '':
            best_fighter_bookie = ''
            best_fighter_bookie_odds = ''
            continue 
        bookie_fighter_odds = row[fighter_col]
        bookies_with_fighter_odds[bookie] = bookie_fighter_odds
    
        opponent_col = f'opponent {bookie}'
        # if the opponent bookie odds are empty, set them to the predicted opponent odds
        if row[opponent_col] == '':
            best_opponent_bookie = ''
            best_opponent_bookie_odds = ''
            continue 
        bookie_opponent_odds = row[opponent_col]
        bookies_with_opponent_odds[bookie] = bookie_opponent_odds
        
    # compute the kelly bankroll for the fighter
    fighter_bookie_kelly_dict = {}
    opponent_bookie_kelly_dict = {}
    for bookie in bookies_with_fighter_odds.keys():
        bookie_fighter_odds = bookies_with_fighter_odds[bookie]
        bookie_opponent_odds = bookies_with_opponent_odds[bookie]
        if bookie_opponent_odds == '':
            print(f'weird, no opponent odds for {bookie} in row {i}')
            continue 
        
        # get the kelly bet for the fighter and opponent
        fighter_kelly, opponent_kelly = get_kelly_bet_from_ev_and_dk_odds(int(predicted_fighter_odds), int(bookie_fighter_odds), int(bookie_opponent_odds))
        fighter_bookie_kelly_dict[bookie] = fighter_kelly
        opponent_bookie_kelly_dict[bookie] = opponent_kelly
        
    best_fighter_bookie = max(fighter_bookie_kelly_dict, key=fighter_bookie_kelly_dict.get)
    best_opponent_bookie = max(opponent_bookie_kelly_dict, key=opponent_bookie_kelly_dict.get)
    prediction_history.at[i, 'best fighter bookie'] = best_fighter_bookie
    prediction_history.at[i, 'best opponent bookie'] = best_opponent_bookie
    prediction_history.at[i, 'fighter bet bankroll percentage'] = fighter_bookie_kelly_dict[best_fighter_bookie]
    prediction_history.at[i, 'opponent bet bankroll percentage'] = opponent_bookie_kelly_dict[best_opponent_bookie]

In [ ]:
# # change settings to show entire dataframe
# import pandas as pd
# pd.set_option('display.max_colwidth', None) 
# pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

In [ ]:
prediction_history[['best fighter bookie', 'best opponent bookie', 'fighter bet bankroll percentage', 'opponent bet bankroll percentage'] ]

In [9]:
ufc_fights_crap = dh.get('ufc_fights_crap')

In [ ]:
updated_prediction_history = dh.update_prediction_correctness(prediction_history, ufc_fights_crap, currentBankroll=300)

searching through 0 confirmed fights on NaT for Amanda Lemos vs Jessica Andrade
fight from NaT between Amanda Lemos and Jessica Andrade not found in ufc_fights_crap.csv
searching through 0 confirmed fights on NaT for Claudio Puelles vs Clay Guida
fight from NaT between Claudio Puelles and Clay Guida not found in ufc_fights_crap.csv
searching through 0 confirmed fights on NaT for Maycee Barber vs Montana de La Rosa
fight from NaT between Maycee Barber and Montana de La Rosa not found in ufc_fights_crap.csv
searching through 0 confirmed fights on NaT for Charles Jourdain vs Lando Vannata
fight from NaT between Charles Jourdain and Lando Vannata not found in ufc_fights_crap.csv
searching through 0 confirmed fights on NaT for Marc-Andre Barriault vs Jordan Wright
fight from NaT between Marc-Andre Barriault and Jordan Wright not found in ufc_fights_crap.csv
searching through 0 confirmed fights on NaT for Sergey Khandozhko vs Dwight Grant
fight from NaT between Sergey Khandozhko and Dwight G

C:\Users\Alex\OneDrive\Documents\GitHub\UFC_Prediction_2022\src\data_handler\data_handler.py:816: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '60.978835978835974' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  vegas_odds_old.at[index1, 'fighter bet'] = fighter_bet
C:\Users\Alex\OneDrive\Documents\GitHub\UFC_Prediction_2022\src\data_handler\data_handler.py:826: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '239.02116402116403' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  vegas_odds_old.at[index1, 'current bankroll after'] = currentBankroll
C:\Users\Alex\OneDrive\Documents\GitHub\UFC_Prediction_2022\src\data_handler\data_handler.py:821: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future ve

In [11]:
updated_prediction_history

,fighter name,fighter DraftKings,fighter BetMGM,fighter Caesars,fighter BetRivers,fighter FanDuel,fighter PointsBet,fighter Unibet,fighter Bet365,fighter BetWay,...,correct?,date,fighter bet bankroll percentage,opponent bet bankroll percentage,fighter bet,opponent bet,current bankroll after,bet result,best fighter bookie,best opponent bookie
0,Ilia Topuria,-470,-525,-470,-480,-500,,,,,...,1,2025-06-28,-46.153846,6.072874,0.00000,10.405269,160.934828,L,DraftKings,BetMGM
1,Alexandre Pantoja,-245,-275,-250,-295,-265,,,,,...,1,2025-06-28,-0.877193,-2.923977,0.00000,0.000000,171.340097,N/A,DraftKings,BetRivers
2,Brandon Royval,+102,+100,+100,-106,+102,,,,,...,1,2025-06-28,0.000000,-8.910891,0.00000,0.000000,171.340097,N/A,DraftKings,BetMGM
3,Beneil Dariush,-102,+100,-110,-105,-104,,,,,...,0,2025-06-28,-49.622166,47.103275,0.00000,152.574277,171.340097,L,BetMGM,Caesars
4,Payton Talbott,+160,+165,+160,+155,+150,,,,,...,0,2025-06-28,11.093074,-29.464286,30.37276,0.000000,323.914373,W,BetMGM,Caesars
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
772,John Castaneda,,,,,,,,,,...,0,2023-06-03,NaN,NaN,0.00000,0.000000,300.000000,N/A,NaN,NaN
773,Jamie Mullarkey,,,,,,,,,,...,0,2023-06-03,NaN,NaN,0.00000,0.000000,300.000000,N/A,NaN,NaN
774,Jinh Yu Frey,,,,,,,,,,...,1,2023-06-03,NaN,NaN,0.00000,0.000000,300.000000,N/A,NaN,NaN
775,Da'Mon Blackshear,,,,,,,,,,...,1,2023-06-03,NaN,NaN,0.00000,0.000000,300.000000,N/A,NaN,NaN
